In [1]:
### raw maxquant search results processing for model training
## 读取数据
import pickle
import pandas as pd
file_name = r"C:\Users\deep\Desktop\PepMTL\fine_tune_data\PXD019086_HeLa_trypsin.pkl"
with open(file_name, 'rb') as f:
   Raw_data = pickle.load(f)
pd.set_option('display.max_columns', None)
Raw_data

,Raw file,Sequence,Length,Modified sequence,Charge,Fragmentation,Mass analyzer,Type,m/z,Mass,Mass error [ppm],Mass error [Da],Retention time,Score,Matches,Intensities,Mass Deviations [Da],Mass Deviations [ppm],Masses,Number of Matches,Intensity coverage,Peak coverage,Fraction,K0,MS/MS scan number,Intensity,CCS
0,20190504_TIMS1_FlMe_SA_HeLa_frac02_B10_1_94,AAAAAAAAAAAAAAAGAGAGAK,22,_AAAAAAAAAAAAAAAGAGAGAK_,3.0,HCD,TOF,TIMS-MULTI-MSMS,532.9533,1595.8380,0.5552,0.0002959,70.32,78.86,y3;y4;y5;y7;y8;y9;y10;y11;y12;y14;y15;b3;b4;b5...,54.7;92.4;223.1;292.9;198.1;244.8;207.3;49.8;3...,0.000248361115609441;0.000680360032561111;-8.2...,0.902569657696691;1.96517814696182;-0.20341690...,275.171133320884;346.207815109767;403.23004121...,21,0.551500838531782,0.233333333333333,2,0.941929696178109,52030,43961.0,567.54
1,20190504_TIMS1_FlMe_SA_HeLa_frac13_E11_1_105,AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGP...,52,_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVG...,3.0,HCD,TOF,TIMS-MULTI-MSMS,1468.0429,4401.1069,-1.7447,-0.0025613,84.68,63.10,y2;y3;y4;y5;y6;y7;y8;y9;y10;y12;y13;y14;y16;y1...,131.5;1862.8;659.9;116.2;77.4;1166.8;3220.6;93...,-0.00132621322291016;0.000215915721810234;0.00...,-4.36020746520311;0.597801794393272;2.29822629...,304.162871490423;361.182793085078;460.25036515...,35,0.42921293465331,0.202312138728324,13,1.41422882878551,92244,92636.0,840.61
2,20190504_TIMS1_FlMe_SA_HeLa_frac20_D12_1_112,AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGP...,52,_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVG...,3.0,HCD,TOF,TIMS-MULTI-MSMS,1468.0429,4401.1069,-1.0221,-0.0015005,84.68,55.18,y2;y3;y4;y5;y7;y8;y9;y10;y12;y13;y14;y16;y18;b...,105.4;1600;559;158.2;1123.8;3099.7;910.1;426.1...,-0.000515113470498818;-0.000219682742454097;0....,-1.6935493856045;-0.608230739989802;4.15088303...,304.16206039067;361.183228683542;460.249512475...,33,0.428992003248536,0.210191082802548,20,1.44752421751816,84951,209040.0,840.61
3,20190504_TIMS1_FlMe_SA_HeLa_frac11_C11_1_103,AAAAAAAAAAGAAGGR,16,_AAAAAAAAAAGAAGGR_,2.0,HCD,TOF,TIMS-MULTI-MSMS,599.8180,1197.6214,-0.67093,-0.00040244,32.52,143.39,y3;y4;y5;y6;y7;y8;y9;y10;y11;y12;y13;y14;y3-NH...,28.1;20.1;20.3;127;126.2;239.5;166.5;329.2;217...,-0.00559477064166458;0.00331704461677873;-0.00...,-19.347855955431;9.2090073101229;-3.9394102551...,289.167474398842;360.195676371383;431.23780602...,27,0.590848329561682,0.296703296703297,11,1.01544537090016,25423,13071.0,397.97
4,20190504_TIMS1_FlMe_SA_HeLa_frac16_H11_1_108,AAAAAAAAAAGAAGGR,16,_(ac)AAAAAAAAAAGAAGGR_,2.0,HCD,TOF,TIMS-MULTI-MSMS,620.8233,1239.6320,6.3307,0.0039302,71.45,151.22,y2;y3;y4;y5;y6;y7;y8;y9;y10;y11;y12;y13;y14;y2...,14.7;53.4;80.3;97.6;383.8;396.2;507.5;846.9;80...,-0.00103597183687043;0.00111818536106512;-0.00...,-4.46267492727603;3.86700240892179;-1.29292346...,232.141451876437;289.160761442839;360.19945912...,28,0.661297362998988,0.282828282828283,16,1.0368607035651,27744,55383.0,409.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249816,20190504_TIMS1_FlMe_SA_HeLa_frac13_E11_1_105,YYVTIIDAPGHRDFIK,16,_YYVTIIDAPGHRDFIK_,3.0,HCD,TOF,TIMS-MULTI-MSMS,636.6720,1906.9941,-0.46003,-0.00029289,74.40,84.31,y3;y5;y6;y7;y8;y9;y10;y11;y12;y13;y8-H2O;y9-H2...,153.9;292.6;110.1;360.7;1441.8;592.4;575.6;194...,0.00187161687853177;0.00596496529988144;0.0033...,4.59559300076153;8.79286017613231;4.1448837559...,407.263410450322;678.387371162;815.44886804893...,24,0.280087962519387,0.137931034482759,13,0.905118534764556,67083,212860.0,532.57
249817,20190504_TIMS1_FlMe_SA_HeLa_frac15_G11_1_107,YYYAVVDCDSPETASK,16,_YYYAVVDCDSPETASK_,2.0,HCD,TOF,TIMS-MULTI-MSMS,934.4064,1866.7982,-1.5489,-0.0014473,60.42,50.12,y3;y4;y6;y7;y8;y9;y10;y11;y12;y12-H2O;a2;b2;b3...,29.9;41.2;170.1;44.9;22.3;65.2;89.6;95;12.7;14...,5.16237979582002E-05;0.00582693913435151;0.017...,0.169157472468737;14.3441599542602;27.89648119...,305.181894744502;406.223797903266;632.30734264...,16,0.310231382181189,0.123076923076923,1

In [2]:
##删除建模不需要的数据
Data = Raw_data[['Modified sequence','Charge','Length','Fragmentation','m/z','Mass','Retention time','CCS','Matches','Intensities','Masses','Intensity','Score']]
##查看数据中所有的修饰类型
import re
Mod_seq = Data.loc[:,'Modified sequence'].str.replace('_', '')
All_AA = Mod_seq.str.cat()
result = re.split(r'(\(.*?\)|[\w])', All_AA)
result = [x for x in result if x]
AA_type = set(result)
print(AA_type)
## 可以发现数据中只包含'(ac)'以及'(ox)'两种可变修饰类型

{'T', '(ox)', 'D', 'H', 'K', 'M', 'Q', 'P', 'V', 'L', 'S', 'I', 'C', 'Y', 'E', 'G', 'A', '(ac)', 'N', 'R', 'F', 'W'}


In [3]:
## 将修饰字符替换成符合unimod的格式
Data.loc[:,'Modified sequence'] = Data['Modified sequence'].str.replace('_', '')
Data.loc[:,'Modified sequence'] = Data['Modified sequence'].str.replace('(ac)', '[1]')
Data.loc[:,'Modified sequence'] = Data['Modified sequence'].str.replace('(ox)', '[35]')
Data.loc[:,'Modified sequence'] = Data['Modified sequence'].str.replace('C', 'C[4]')

In [4]:
##将Fragmentation替换成对应的编号
FTdict = {'CID': 1, 'HCD': 2, 'ECD': 3, 'ETD': 4, 'EAD': 5, 'EThcD': 6, 'ETciD': 7, 'UVPD': 8}  
def convert_frag(x):
    return FTdict.get(x)
Data.loc[:,'Fragmentation'] = Data['Fragmentation'].apply(convert_frag)

In [5]:
##删除多肽序列长度超过72的数据
Data.loc[:,'Length'] = Data['Length'].astype(int)
Data = Data.query('Length <= 72')

In [6]:
##读入Maxquant的离子编号作为字典
Peaks = pd.read_excel(r"C:\Users\deep\Desktop\PepMTL\fine_tune_data\Ion_index\Maxquant_ion_index.xlsx")
Peaks_dict = dict(zip(Peaks.ion,Peaks.num))

import numpy as np
from tqdm import tqdm
##利用for循环处理二级质谱数据，将其制备成模型需要的格式
PI_inten = []
PI_mass = []
for i in tqdm(range(len(Data))):
    datai = Data.loc[i,:]
    Ion_name = datai['Matches'].split(';')
    Ion_inten = datai['Intensities'].split(';')
    Ion_mass = datai['Masses'].split(';')
    ##建立一个空PI_intensityi用于存放匹配后的碎片响应值, ##建立一个空PI_massi用于存放匹配后的碎片质荷比
    PI_intensityi = np.zeros(len(Peaks_dict))
    PI_massi = np.zeros(len(Peaks_dict))
    ##利用Peaks_dict将Ioni中对应名称的离子碎片的响应值放入到PI_intensityi的固定位置，有序列覆盖但是无离子碎片相应的位置为0
    for j in range(len(Ion_name)):
        Ionii = Ion_name[j]
        if Ionii in Peaks_dict:
           indexii = Peaks_dict[Ionii]
           PI_intensityi[indexii] = Ion_inten[j]
           PI_massi[indexii] = Ion_mass[j]
    ##保存每个循环的数据
    PI_inten.append(PI_intensityi); PI_mass.append(PI_massi)
#将整理后的结果替换原先的数据
Data['Intensities'] = PI_inten
Data['Masses'] = PI_mass
Data

100%|███████████████████████████████████████████████████████████████████████| 249821/249821 [00:20<00:00, 12139.15it/s]


,Modified sequence,Charge,Length,Fragmentation,m/z,Mass,Retention time,CCS,Matches,Intensities,Masses,Intensity,Score
0,AAAAAAAAAAAAAAAGAGAGAK,3.0,22,2,532.9533,1595.8380,70.32,567.54,y3;y4;y5;y7;y8;y9;y10;y11;y12;y14;y15;b3;b4;b5...,"[0.0, 0.0, 105.8, 258.0, 447.1, 418.0, 438.4, ...","[0.0, 0.0, 214.119643974358, 285.157343718957,...",43961.0,78.86
1,AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGP...,3.0,52,2,1468.0429,4401.1069,84.68,840.61,y2;y3;y4;y5;y6;y7;y8;y9;y10;y12;y13;y14;y16;y1...,"[0.0, 0.0, 54.5, 163.1, 388.9, 912.3, 1266.1, ...","[0.0, 0.0, 214.117024092357, 285.155549934846,...",92636.0,63.10
2,AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGP...,3.0,52,2,1468.0429,4401.1069,84.68,840.61,y2;y3;y4;y5;y7;y8;y9;y10;y12;y13;y14;y16;y18;b...,"[0.0, 0.0, 17.9, 98.4, 358.1, 575.6, 1189.0, 1...","[0.0, 0.0, 214.117562639275, 285.155775087783,...",209040.0,55.18
3,AAAAAAAAAAGAAGGR,2.0,16,2,599.8180,1197.6214,32.52,397.97,y3;y4;y5;y6;y7;y8;y9;y10;y11;y12;y13;y14;y3-NH...,"[0.0, 0.0, 54.2, 108.7, 100.6, 79.9, 53.6, 49....","[0.0, 0.0, 214.118524618109, 285.156051001449,...",13071.0,143.39
4,[1]AAAAAAAAAAGAAGGR,2.0,16,2,620.8233,1239.6320,71.45,409.09,y2;y3;y4;y5;y6;y7;y8;y9;y10;y11;y12;y13;y14;y2...,"[0.0, 0.0, 624.3, 1008.1, 666.0, 456.4, 248.0,...","[0.0, 0.0, 256.129507258295, 327.166322347601,...",55383.0,151.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
249816,YYVTIIDAPGHRDFIK,3.0,16,2,636.6720,1906.9941,74.40,532.57,y3;y5;y6;y7;y8;y9;y10;y11;y12;y13;y8-H2O;y9-H2...,"[0.0, 1289.3, 211.3, 175.1, 0.0, 0.0, 0.0, 0.0...","[0.0, 327.133358274998, 426.202562281217, 527....",212860.0,84.31
249817,YYYAVVDC[4]DSPETASK,2.0,16,2,934.4064,1866.7982,60.42,452.93,y3;y4;y6;y7;y8;y9;y10;y11;y12;y12-H2O;a2;b2;b3...,"[0.0, 171.4, 180.6, 200.5, 176.7, 23.1, 0.0, 0...","[0.0, 327.133940305433, 490.194982538756, 561....",61190.0,50.12
249818,YYYAVVDC[4]DSPETASK,2.0,16,2,934.4064,1866.7982,60.42,452.93,y2;y3;y4;y6;y7;y8;y9;y10;y11;y12;y13;y14;y6-H2...,"[0.0, 361.6, 273.3, 180.5, 151.9, 0.0, 0.0, 0....","[0.0, 327.134565248805, 490.196021901735, 561....",54186.0,69.64
249819,YYYAVVDC[4]DSPETASK,2.0,16,2,934.4064,1866.7982,60.42,452.93,y2;y3;y4;y6;y7;y8;y9;y10;y11;y12;y13;y14;y9-H2...,"[0.0, 1319.7, 1584.1, 1775.7, 1069.7, 330.1, 6...","[0.0, 327.134201394229, 490.194702307244, 561....",2715800.0,145.08


In [7]:
### 将数据保存为pickle文件
with open(r"C:\Users\deep\Desktop\PepMTL\fine_tune_data\Modeling_data.pkl", 'wb') as f:
    pickle.dump(Data, f)